# Critical Role Cosine Similarity Attempt 1

I have been binge watching critical role campaign 3 since I haven’t watched a few episodes and while we were discussing cosine similarity and nlp I decided that it would be interesting to see if my favorite actor in the campaign was good at choosing different vocabulary for her characters. 

Laura Bailey was my first guinea pig. She has 3 characters: Vex from campaign 1, Jester from campaign 2, and Imogen from campaign 3. Each of these characters seem to have different personalities and vocab, but I wanted to put it up to a little test to see if they were really that different. 

I didn’t have a lot of time to gather the data, but that goodness there are some dedicated critters out there typing the closed captions of each show out and letting others use it. I was able to grab 3 paragraphs of each character speaking and put them to the test. Some of the notable quotes are: Vex resurrecting Percy with her love confession, Jester’s opening to Traveler Con, and Imogen talking to Laudna about dreaming of Bertrand’s death. 

I know 3 paragraphs isn’t enough and this wasn’t supposed to be a huge science experiment, but baby steps here.


In [1]:
# Locally (or on colab) let's use en_core_web_lg 
!python -m spacy download en_core_web_md # Can do lg, takes awhile
# Also on Colab, need to restart runtime after this step!

     |████████████████████████████████| 96.4 MB 1.3 MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.2.5-py3-none-any.whl size=98051301 sha256=9914fac351be6f0281810ab4c93b6d4df1544db4183d28653876adef1a9babb0
  Stored in directory: /tmp/pip-ephem-wheel-cache-ikf59uj2/wheels/69/c5/b8/4f1c029d89238734311b3269762ab2ee325a42da2ce8edb997
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [1]:
Laura_Imogen_1 = ''''''
Laura_Imogen_2 = '''Yeah. →
Something happened in it, though. →
I saw someone else. →
I think it was Bertrand? I don't know what it means, but it's never happened. I don't know. →
He looked almost like he was at peace, but I don't understand how that's possible. →
Laudna, I swear that the storm is terrifying. →
Nothing, nothing would be comfortable there. →
I don't know-- Yeah, it's got to mean something.'''
Laura_Imogen_3 = '''No, I mean, so there was a boy. He was also-- he worked in the stables, so- Well, you know, we flirted. →
It was young. →
It was sweet. →
We didn't go on any dates, per se, but-- Made eyes. No, none of that, but man, I wish it would've happened. Sam. →
Samuel. He's quite happy. →
He's fine I don't think he's... →
No, he wasn't married. →
At least not when we left. I didn't talk to him for a long time. →
He was fine. →
He seemed happy.'''
Laura_Jester_1 = '''It's kind of a wait list, but usually she only has, you know, a few patrons during any given period of time. So she'll have the best diplomats that come to town and if they've claimed that time then they can spend time with her for an entire week or even more and nobody else will come to see her. You know what I mean? Sometimes. There's a lot of people that want to marry her, and have tried to court her and stuff like that. But she only ever, you know, said yes to my dad. Yeah, there's lot of security guards around here, unfortunately. It would usually-- you know, she'd come tuck me into bed, late at night, afterwards. →
Sometimes I would already be asleep, but she'd come in and say, "I'm just saying good night to "you, love." Then she would go, and then we'd wake up, and sometimes we'd have breakfast, but sometimes we wouldn't and then sometimes she was gone for a few days at a time because the people here, they don't really want to know about their daughters or anything like that. So I would stay hidden, she would tell me to, and then sometimes she'd be free and then we could have lunch or something.'''
Laura_Jester_2 = '''I did, I did. I'm really sorry about it. 
You know, we just really needed to see, like, who you were going to come to see, and I didn't know it would bother you. →
You know, I didn't even think about the fact that it's like, you know, in the future, if you ever want to scry on us, I wouldn't blame you, you know, because I did it to you before'''
Laura_Jester_3 = '''Welcome, one and all, to Traveler Con 3000! (cheering) Thank you. →
As you know, the Traveler likes everyone here. →
Me mostly, but the Traveler's been hard to work for all of you hiding many, many dicks around the village and the island. →
Once everybody finds one, we're going to make a special trek just so we can help the Traveler fully ascend and you're going to see some crazy shit, you guys. →
(laughter) (cheering) → Whoever finds the-- (whispering) Did we end up--? Whoever finds the golden dick gets a special prize. Once you guys find them, come up to the stage. →
And once everybody's here, we'll start the rest of the shit. →
Have fun! Let chaos reign!'''
Laura_Vex_1 = '''How did you know about us? Oh my gosh, Zahra! She's so wonderful. I love her so much! Yes. Any watcher that comes after you we'll fight them. We're very strong right now and full of health. Your guests are vampires. Look at my brother right now. Look at his neck. You invited vampires into your palace.'''
Laura_Vex_2 = '''It's possible the key is now inside of a giant's stomach. In which case, it still would be inside of someone. Just a different someone. I mean and look out for a home in the forest? We'll have to fly pretty low. I'm assuming the home is going to be covered by treetop. It's very far. We could Misty Step to the forest and then walk in the forest? No reason, darling. Don't be dark and creepy and make it sound like you're being nice. I know you're being dark and creepy right now.'''
Laura_Vex_3 = '''Percy, I don't know if you can hear me, but that day in Syngorn was one of the best moments of my life. And not because of what you think. It wasn't because of my father or the title. It was because of Whitestone. It was because of you. You allowed me to be a part of the thing that you hold dearest. And I was so proud. →
But I don't want to be here if you're not. →
Whitestone still needs you, darling. I still need you here. I should have told you: it's yours.'''

corpus = [Laura_Imogen_1 +' '+ Laura_Imogen_2 +' '+ Laura_Imogen_3, Laura_Jester_1 +' '+ Laura_Jester_2 +' '+ Laura_Jester_3,Laura_Vex_1 +" "+ Laura_Vex_2 +" "+ Laura_Vex_3]

#### Clean_data

In [4]:
#Clean data
import re
   
def clean_data(text):
    """
    Accepts a single text document and performs several regex substitutions in order to clean the document. 
    
    Parameters
    ----------
    text: string or object 
    
    Returns
    -------
    text: string or object
    """
    
    # order of operations - apply the expression from top to bottom
    date_regex = r"\d+/\d+/\d+"
    punct_regex = r"[^0-9a-zA-Z\s]" # any non-alphanumeric chars
    special_chars_regex = r"[\$\%\&\@+]" 
    numerical_regex =  r"\d+"  # match one or more digits
    
    # Replace any strings matching the above regex patterns with blank strings
    # (effectively removing them from the text)
    text = re.sub(date_regex, "", text)
    text = re.sub(punct_regex, "", text)
    text = re.sub(special_chars_regex, "", text)
    text = re.sub(numerical_regex, "", text)

    # match one or more whitespace chars
    whitespace_regex = r"\s+"
    # replace one or more whitespace characters with a single white space char
    #    not a blank string!
    text = re.sub(whitespace_regex," ",text)


    # apply case normalization
    text = text.lower()
    return text



import spacy
import en_core_web_md

nlp = en_core_web_md.load()

def spacy_tokenizer(text):
    """
    Use the pre-trained model from Space to tokenize our text into lemmas 
    
    Notes
    -----
    Rememeber that the pre-trained spaCy model has a lot of built in flags for what kind of token each token is
    so we can use that functionality to create filters for stop words, white spaces, punctuation, and so on!
    
    See list of flags here: https://spacy.io/api/token#attributes
    
    Parameter
    ---------
    text: string
        Full text article/document that needs to be tokenized 
    """
    tokens = []
    for token in nlp(text):
        # if statement will filter out stopwords, punctuation, and whitespace
        # COMPLETE THE CODE HERE
        if (token.is_stop == False) & (token.is_punct == False) & (token.text.isspace() == False):
            # Now lemmatize!
            tokens.append(token.text.lower())
    
    return tokens

In [5]:
corpus_clean = [" ".join(spacy_tokenizer(clean_data(corpus[0])))," ".join(spacy_tokenizer(clean_data(corpus[1])))," ".join(spacy_tokenizer(clean_data(corpus[2])))]

#### Count Vector

In [6]:
# Frequency-count

# Import the feature_extraction module and vectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Instantiate the object and count the words
vectorizer = CountVectorizer()
vectors = vectorizer.fit_transform(corpus_clean)

# Put frequency count into DataFrame
import pandas as pd
df_fc = pd.DataFrame(vectors.todense(),columns = vectorizer.get_feature_names_out())

df_fc.head()

,allowed,ascend,asleep,assuming,bed,bertrand,best,blame,bother,boy,...,whispering,whitestone,wish,wonderful,work,worked,yeah,yes,young,zahra
0,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,1,2,0,1,0
1,0,1,1,0,1,0,1,1,1,0,...,1,0,0,0,1,0,1,1,0,0
2,1,0,0,1,0,0,1,0,0,0,...,0,2,0,1,0,0,0,1,0,1


#### One-hot encoded vector


In [7]:
# One-hot encoding of word counts

# Import the binary encoder
from sklearn.preprocessing import Binarizer

# Initialize the vectorizer and get the word counts
freq   = CountVectorizer()
corpus_freq = freq.fit_transform(corpus_clean)

# Initialize the binarizer and create the binary encoded vector
onehot = Binarizer()
corpus_onehot = onehot.fit_transform(corpus_freq.toarray())

# Put frequency count into DataFrame
import pandas as pd
df_fc = pd.DataFrame(corpus_onehot,columns = freq.get_feature_names_out())

df_fc.head()

,allowed,ascend,asleep,assuming,bed,bertrand,best,blame,bother,boy,...,whispering,whitestone,wish,wonderful,work,worked,yeah,yes,young,zahra
0,0,0,0,0,0,1,0,0,0,1,...,0,0,1,0,0,1,1,0,1,0
1,0,1,1,0,1,0,1,1,1,0,...,1,0,0,0,1,0,1,1,0,0
2,1,0,0,1,0,0,1,0,0,0,...,0,1,0,1,0,0,0,1,0,1


#### Td-idf

In [8]:
# Import libraries and modules
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Instantiate vectorizer object
tfidf = TfidfVectorizer(stop_words='english', max_features=5000)

# Create a vocabulary and get word counts per document
dtm = tfidf.fit_transform(corpus_clean)

# Get feature names to use as dataframe column headers
dtm_Laura = pd.DataFrame(dtm.todense(), columns=tfidf.get_feature_names())


# Create a Python list
list = ['Imogen', 'Jester', 'Vex']
 
# Create a DataFrame index object
# using pd.Index() function
idx = pd.Index(list)
 
# Set the above DataFrame index object as the index
# using set_index() function
dtm_Laura = dtm_Laura.set_index(idx)

# View feature matrix as DataFrame
dtm_Laura.head()

/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,allowed,ascend,asleep,assuming,bed,bertrand,best,blame,bother,boy,...,whispering,whitestone,wish,wonderful,work,worked,yeah,yes,young,zahra
Imogen,0.000000,0.000000,0.000000,0.000000,0.000000,0.111046,0.000000,0.000000,0.000000,0.111046,...,0.000000,0.000000,0.111046,0.000000,0.000000,0.111046,0.168907,0.000000,0.111046,0.000000
Jester,0.000000,0.063149,0.063149,0.000000,0.063149,0.000000,0.048026,0.063149,0.063149,0.000000,...,0.063149,0.000000,0.000000,0.000000,0.063149,0.000000,0.048026,0.048026,0.000000,0.000000
Vex,0.092753,0.000000,0.000000,0.092753,0.000000,0.000000,0.070541,0.000000,0.000000,0.000000,...,0.000000,0.185505,0.000000,0.092753,0.000000,0.000000,0.000000,0.070541,0.000000,0.092753


In [9]:
# Find the cosine similarity of tf-idf vectors
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim_Laura  = cosine_similarity(dtm_Laura)

# Turn it into a DataFrame
cosine_sim_Laura = pd.DataFrame(cosine_sim_Laura,columns = idx)

cosine_sim_Laura = cosine_sim_Laura.set_index(idx)
display(cosine_sim_Laura)

,Imogen,Jester,Vex
Imogen,1.000000,0.253437,0.182008
Jester,0.253437,1.000000,0.151037
Vex,0.182008,0.151037,1.000000


In [10]:
from sklearn.neighbors import NearestNeighbors

# Instantiate the nearest neighbors model
nn = NearestNeighbors()

# fit to our document term matrix
nn.fit(dtm_Laura.values)

# sample a doc from a dtm to use as our query point
doc_index = 0
query_doc = dtm_Laura.values[0][None,:] # re-cast as a row vector 

# Query Using the kneighbors method
# NOTE: nn counts the original document as one of the neighbors, so if we want 5 nearest neighbors, we should set n_neightbors=6
neigh_dist, neigh_index = nn.kneighbors(query_doc,n_neighbors=3)

In [11]:
# nearest neighbor search returns the same documents as cosine similarity!
print(neigh_index)
print(neigh_dist)

[[0 1 2]]
[[0.         1.22193539 1.27905576]]
